<h1 style="background-color:skyblue;font-family:Georgia;font-size:350%;text-align:center">Restaurant Business Analysis</h1>

<img src="https://media.giphy.com/media/eAH6GS73H0Qxy/giphy.gif">

<h1 style="font-family:Georgia;font-size:150%;text-align:center">we will try through theses datasets to know how restaurants act during Covid-19, what makes difference with sales during covid, people food behaviour during 2020 and how it's affecting on future restaurants </h1>

# Imports 

In [ ]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud, STOPWORDS , ImageColorGenerator
import re
from PIL import Image


In [ ]:
df_250 = pd.read_csv('../input/restaurant-business-rankings-2020/Top250.csv')
df_future = pd.read_csv('../input/restaurant-business-rankings-2020/Future50.csv')
df_ind = pd.read_csv('../input/restaurant-business-rankings-2020/Independence100.csv')


In [ ]:
df_250.info()

<h1 style="background-color:skyblue;font-family:Georgia;font-size:400%;text-align:center">Top 250 Resturants</h1>


<h1 style="font-family:Georgia;font-size:150%;text-align:center">Trying to figure out what happend to Top Resturants Sales During 2020 and Extracting Imformative insights </h1>


<h1 style="background-color:skyblue;font-family:Monaco;font-size:300%;text-align:center">Feature Engineering</h1>

In [ ]:
df_250.drop(['Content','Headquarters'],axis=1,inplace=True)

### Null Values Checikng

In [ ]:
print ("Rows     : " ,df_250.shape[0])
print ("Columns  : " ,df_250.shape[1])
print ("\nFeatures : \n" ,df_250.columns.tolist())
print ("\nMissing values :  ", df_250.isnull().sum().values.sum())
print ("\nUnique values :  \n",df_250.nunique())

### Rename Columns 

In [ ]:
df_250.rename(columns={'Units':'Branches','Segment_Category':'sub_category'},inplace=True)

### Checking for any independence restaurant in top 250 

In [ ]:
rest_ind = list(df_ind['Restaurant'].values)
df_250['ind_100']=df_250['Restaurant'].apply(lambda x:1 if x in rest_ind else 0 )
df_250['ind_100'].value_counts()

### Deal with floats instead of percentages 

In [ ]:
df_250['YOY_Sales']=df_250['YOY_Sales'].apply(lambda x:x.replace('%',''))
df_250['YOY_Units']=df_250['YOY_Units'].apply(lambda x:x.replace('%',''))
#====
df_250['YOY_Sales']= pd.to_numeric(df_250['YOY_Sales'])
df_250['YOY_Units']=pd.to_numeric(df_250['YOY_Units'])
#====

In [ ]:
df_250.describe()

### Manipulate with YOY values 

In [ ]:
df_250['Sales_year']=df_250['YOY_Sales'].apply(lambda x:'positive' if x>0 else 'negative')
df_250['unit_stat']=df_250['YOY_Units'].apply(lambda x:'positive' if x>0 else 'negative')

### Making Category Feature 

In [ ]:
df_250['category']=df_250['sub_category'].apply(lambda x:x.replace('Quick Service & Burger','Burger').replace(
                                                      'Italian/Pizza','Pizza').replace('Quick Service & Pizza','Pizza').replace(
'Bakery Cafe ','Cafe').replace('Coffee Cafe','Cafe')).replace('Quick Service & Chicken','Chicken').replace('Casual Dining & Pizza','Pizza').replace(
'Quick Service & Cafe','Cafe').replace('Fast Casual & Pizza','Pizza').replace('Fast Casual & Bakery Cafe','Cafe').replace('Quick Service & Frozen Desserts','Dessert').replace(
'Quick Service & Family Casual','Family').replace('Casual Dining & Asian','Asian').replace('Asian/Noodle','Asian').replace('Casual Dining & Seafood','Seafood').replace(
'Bakery Cafe','Cafe').replace('Frozen Desserts','Dessert').replace('Family Dining & Family Style','Family').replace('Fast Casual & Sandwich','Sandwich').replace(
'Family Casual','Family').replace('Fast Casual & Chicken','Chicken').replace('Fast Casual & Burger','Burger').replace('Casual Dining & Steak','Steak').replace(
'Casual Dining & Sports Bar','Sports Bar').replace('Quick Service & Mexican','Mexican').replace('Quick Service & Mexican','Mexican').replace(
'Quick Service & Seafood','Seafood').replace('Quick Service & Sandwich','Sandwich').replace('Seafodd','Seafood').replace('Fine Dining & Steak','Steak').replace(
'Quick Service & Bakery Cafe','Cafe').replace('Fast Casual & Asian/Noodle','Asia').replace('Quick Service & Snack','Snack').replace(
'Fast Casual & Mexican','Mexican').replace('Quick Service & Beverage','Drinks').replace('Asian','Asia').replace('Sports Bar','Sports Bar').replace(
'Casual Dining & Varied Menu','Varied Menu').replace('Steak','Meat').replace('BBQ','Meat').replace('Family Style','Family')

In [ ]:
df_250.head()

<h1 style="background-color:skyblue;font-family:Monaco;font-size:300%;text-align:center">EDA-Data Visualization</h1>

<h1 style="color:white;background-color:red;font-family:Monaco;font-size:200%;text-align:left">YOY Distribution</h1>  

In [ ]:
plt.figure(figsize=(14,7))
sns.kdeplot(df_250['YOY_Sales'].values, shade = True)
sns.kdeplot(df_250['YOY_Units'].values, shade = True)
plt.legend(['Sales','Units'])
plt.xlabel('Percentage');

<h1 style="font-family:Georgia;font-size:150%;text-align:center">These Distributions shows us that year on year sales percentage no more than 40% </h1>


<h1 style="color:white;background-color:green;font-family:Monaco;font-size:200%;text-align:left">Correlation With Sales </h1>   

In [ ]:
corr = df_250.corr().loc[:,['Sales']]

fig, ax = plt.subplots(figsize=(5,10))

ax = sns.heatmap(corr.sort_values(by=['Sales'],ascending=False), ax=ax,annot=True,cbar=True,cmap="Greens")
ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=10)
ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=10)

plt.show()

<h1 style="font-family:Georgia;font-size:150%;text-align:center">Sales High Correlated with Branches </h1>


<h1 style="color:white;background-color:blue;font-family:Monaco;font-size:200%;text-align:left"> YOY Sales Indicator </h1>   

In [ ]:
plt.figure(figsize=(14,7));
Platform = df_250['Sales_year'].value_counts(sort = True).index
Count = df_250['Sales_year'].value_counts(sort = True)
#====
fig = px.pie(names = Platform,
             values = Count,
             title='Positive - Negative')
fig.update_traces(textposition='inside', textinfo='percent+label')

<h1 style="font-family:Georgia;font-size:150%;text-align:center">Although the restaurants were on the best list during the year, about 35% of restaurants had negative indicators </h1>



<h1 style="color:white;background-color:tomato;font-family:Monaco;font-size:200%;text-align:left"> Restaurant Category and Sub Category  </h1> 

In [ ]:
plt.figure(figsize=(40,40));

df = px.data.tips()
fig = px.sunburst(df_250, path=['category', 'sub_category'], values='Sales')
fig.show()

<h1 style="font-family:Georgia;font-size:150%;text-align:center">sunburst shows that quick-service subcategory always get high sales  </h1>

<h1 style="color:black;background-color:skyblue;font-family:Monaco;font-size:300%;text-align:center">Category Sales Indicator  </h1>  

In [ ]:
plt.figure(figsize=(20,7))
#=== 
pal = ["#58D3F7","#FA5858"]
#=== 
sns.barplot(data=df_250,x=df_250['category'],y=df_250['Sales'],hue=df_250['Sales_year'],palette=pal);

<h1 style="font-family:Georgia;font-size:175%;text-align:center">varied menu, sandwiches and sports bar had negative indicators more than positive</h1>


<h1 style="color:white;background-color:indigo;font-family:Monaco;font-size:200%;text-align:left">Top Resturants In Best Category  </h1> 

## Pizza 🍕

In [ ]:
pizza = df_250.query("category == 'Pizza'")[:10]

fig = px.bar(pizza,
             x='Restaurant',
             y='Sales',
             color='Sales',
             color_continuous_scale='Viridis',
             title='Pizza Sales Top Restaurants')
fig.show()


## Burger 🍔

In [ ]:
Burger = df_250.query("category == 'Burger'")[:10]

fig = px.bar(Burger,
             x='Restaurant',
             y='Sales',
             color='Sales',
             color_continuous_scale='BlackBody',
             title='Burger Top Restaurants')
fig.show()


## Varied Menu ✔️

In [ ]:
VariedMenu = df_250.query("category == 'Varied Menu'")[:10]

fig = px.bar(VariedMenu,
             x='Restaurant',
             y='Sales',
             color='Sales',
             color_continuous_scale='Plotly3',
             title='VariedMenu Top Restaurants')
fig.show()

## Family 👪

In [ ]:
Family = df_250.query("category == 'Family'")[:10]

fig = px.bar(Family,
             x='Restaurant',
             y='Sales',
             color='Sales',
             color_continuous_scale='Turbo',
             title='Family Top Restaurants')
fig.show()

## Meat 🥩

In [ ]:
Meat = df_250.query("category == 'Meat'")[:10]

fig = px.bar(Meat,
             x='Restaurant',
             y='Sales',
             color='Sales',
             color_continuous_scale='Jet',
             title='Meat Top Restaurants')
fig.show()

## Mexican 🌮

In [ ]:
Mexican = df_250.query("category == 'Mexican'")[:10]

fig = px.bar(Mexican,
             x='Restaurant',
             y='Sales',
             color='Sales',
             color_continuous_scale='Aggrnyl',
             title='Mexican Top Restaurants')
fig.show()

## Cafe ☕

In [ ]:
Cafe = df_250.query("category == 'Cafe'")[:10]

fig = px.bar(Cafe,
             x='Restaurant',
             y='Sales',
             color='Sales',
             color_continuous_scale='Darkmint',
             title='Cafe Top Restaurants')
fig.show()

## Sandwich 🥪 

In [ ]:
Sandwich = df_250.query("category == 'Sandwich'")[:10]

fig = px.bar(Sandwich,
             x='Restaurant',
             y='Sales',
             color='Sales',
             color_continuous_scale='Magenta',
             title='Sandwich Top Restaurants')
fig.show()

## Chicken 🐔

In [ ]:
Chicken = df_250.query("category == 'Chicken'")[:10]

fig = px.bar(Chicken,
             x='Restaurant',
             y='Sales',
             color='Sales',
             color_continuous_scale='Brwnyl',
             title='Chicken Top Restaurants')
fig.show()

## Drinks 🍸

In [ ]:
Drinks = df_250.query("category == 'Drinks'")[:10]

fig = px.bar(Drinks,
             x='Restaurant',
             y='Sales',
             color='Sales',
             color_continuous_scale='thermal',
             title='Drinks Top Restaurants')
fig.show()

<h1 style="background-color:skyblue;font-family:Georgia;font-size:350%;text-align:center">Notes</h1>


## 1-Number Branches is The Highest Correlated With Sales  

    
## 2-35% YOY for Top Restaurants are Negative 
       
    
## 3-Burger Restaurants is the highest Sales in 2020     
    
    
## 4-Quick Service is an optimal solution to get high sales
  
    
## 5-Sports Bar and Sandwich Restaurants face Big losses 

<img src="https://media.giphy.com/media/3kyuaA4dDSPKi2F4J8/giphy.gif">

<h1 style="background-color:skyblue;font-family:Georgia;font-size:400%;text-align:center">Top 50 Future Restaurants</h1>

<h1 style="font-family:Georgia;font-size:200%;text-align:center">Study High Sales indicates </h1>

<h1 style="color:white;background-color:dodgerblue;font-family:Georgia;font-size:200%;text-align:left">Top Commin' Restaurant Study</h1>
 

In [ ]:
df_future.head(10)

<h1 style="color:white;background-color:firebrick;font-family:Georgia;font-size:200%;text-align:left">Franchising</h1> 

In [ ]:
Platform = df_future['Franchising'].value_counts(sort = True).index
Count = df_future['Franchising'].value_counts(sort = True)
#====
fig = px.pie(names = Platform,
             values = Count,
             title='Franchising Or Not')
fig.update_traces(textposition='inside', textinfo='percent+label')

<h1 style="color:white;background-color:darkslategray;font-family:Georgia;font-size:200%;text-align:left">Franchising / Sales</h1> 

In [ ]:
df_future['YOY_Sales']=df_future['YOY_Sales'].apply(lambda x:x.replace('%',''))
#====
df_future['YOY_Sales']= pd.to_numeric(df_future['YOY_Sales'])
#===
franch = df_future.groupby('Franchising',as_index=False).agg({'YOY_Sales':'sum'})
fig = px.bar(franch,
             x='Franchising',
             y='YOY_Sales',
             color='YOY_Sales',
             color_continuous_scale='thermal',
             title='Franchising Or not')
fig.show()

<h1 style="color:black;background-color:lightskyblue;font-family:Georgia;font-size:200%;text-align:left">Correlation with YOY </h1> 

In [ ]:
corr = df_future.corr().loc[:,['YOY_Sales']]

fig, ax = plt.subplots(figsize=(5,10))

ax = sns.heatmap(corr.sort_values(by=['YOY_Sales'],ascending=False), ax=ax,annot=True,cbar=True,cmap="Blues")
ax.set_xticklabels(ax.xaxis.get_ticklabels(), fontsize=10)
ax.set_yticklabels(ax.yaxis.get_ticklabels(), fontsize=10)

plt.show()

<h1 style="color:black;background-color:yellowgreen;font-family:Georgia;font-size:400%;text-align:center">Study Future Restaurants Categories</h1>


<h1 style="font-family:Georgia;font-size:275%;text-align:center">Most of them depending on organic food</h1>

<a href='https://postimages.org/' target='_blank'><img src='https://i.postimg.cc/9Q7sbdyB/collage.png' border='0' alt='collage'/></a>

<h1 style="background-color:skyblue;font-family:Georgia;font-size:350%;text-align:center">Notes</h1>


<h1 style="font-family:Georgia;font-size:150%;text-align:center">The demand for organic produce and foods continues to rise across the nation and many consumers are prioritizing nutrition and immunity-boosting ingredients in an effort to protect themselves and their families against COVID-19</h1>

<img src="https://media.giphy.com/media/8JNmK2TcgnkiZAjHcG/giphy.gif">


<h1 style="color:white;background-color:indigo;font-family:Georgia;font-size:400%;text-align:center">Future Restaurants Brand Names words </h1>

In [ ]:
rest = df_future['Restaurant'].drop_duplicates()
#==== 
all_sentences = []

for word in rest:
    all_sentences.append(word)

all_sentences

lines = list()
for line in all_sentences:    
    words = line.split()
    for w in words: 
     lines.append(w)
#=== 


lines = [re.sub(r'[^A-Za-z0-9]+', '', x) for x in lines]

lines

lines2 = []

for word in lines:
    if word != '':
        lines2.append(word)
#== 
from nltk.stem.snowball import SnowballStemmer
s_stemmer = SnowballStemmer(language='english')

stem = []
for word in lines2:
    stem.append(s_stemmer.stem(word))
#== 
import spacy
nlp = spacy.load('en_core_web_lg')
#== 
stem2 = []

for word in stem:
    if word not in nlp.Defaults.stop_words:
        stem2.append(word)
#== 
df = pd.DataFrame(stem2) 
rest_All = " ".join(rest for rest in df[0])
#=== 
c_mask = np.array(Image.open("../input/wordcloudterry/44.jpg"))
#== 
wc = WordCloud(background_color='white', mask=c_mask, mode='RGB',
               width=1000, max_words=1000, height=1000,
               random_state=1, contour_width=1, contour_color='indigo',colormap='inferno')
wc.generate(rest_All)
plt.figure(figsize=(20, 10))
plt.imshow(wc, interpolation='bilinear')
plt.tight_layout(pad=0)
plt.axis('off')
plt.show()

<h1 style="background-color:orange;font-family:Georgia;font-size:350%;text-align:center">
Is diet Related to your Infection with Corona</h1>

<h1 style="font-family:Georgia;font-size:200%;text-align:center">Now we are trying to find if there is a relationship between some food products and the high rates of infection with the Coronavirus
 </h1>

In [ ]:
df_health = pd.read_csv('../input/covid19-healthy-diet-dataset/Food_Supply_Quantity_kg_Data.csv')

<h1 style="color:white;background-color:firebrick;font-family:Georgia;font-size:350%;text-align:center">
Correlations</h1>

In [ ]:
corr = df_health.corr()
plt.figure(figsize=(20,10))
sns.heatmap(corr,annot=True)

<h1 style="font-family:Georgia;font-size:200%;text-align:left"> Manipulate With Features</h1>

In [ ]:
df_health['Alcohol']=df_health['Alcoholic Beverages']
df_health['animal_products']=df_health['Animal Products']+df_health['Eggs']+df_health['Animal fats']+df_health['Meat']+df_health['Fish, Seafood']+df_health['Aquatic Products, Other']+df_health['Milk - Excluding Butter']
df_health['vegan products']=df_health['Fruits - Excluding Wine']+df_health['Oilcrops']+df_health['Pulses']+df_health['Spices']+df_health['Starchy Roots']+df_health['Sugar Crops']+df_health['Treenuts']+df_health['Vegetable Oils']+df_health['Vegetables']+df_health['Vegetal Products']
df_health['manufactured product']=df_health['Stimulants']+df_health['Sugar & Sweeteners']
df_health['other']=df_health['Miscellaneous']+df_health['Offals']

<h1 style="color:vlack;background-color:orange;font-family:Georgia;font-size:350%;text-align:center">Confirmed Cases & Veg_products</h1>

In [ ]:
df_veg=df_health[['Country','Alcohol','animal_products','vegan products','manufactured product','other','Confirmed', 'Deaths',
       'Recovered']]
fig = px.choropleth(df_veg, 
                    locations='Country', locationmode='country names',
                    color='Confirmed', hover_name='vegan products', 
                    color_continuous_scale="Peach", hover_data=['vegan products'],
                    title='Country Confiremed Cases Percentage Hovering with Veg_products')
fig.update(layout_coloraxis_showscale=False)
fig.show();

<h1 style="font-family:Georgia;font-size:200%;text-align:center">we will notice here the negative correlation between confirmed cases and veg_product</h1>

<h1 style="color:black;background-color:aqua;font-family:Georgia;font-size:400%;text-align:center">VEGAN VS ANIMAL 
 </h1>

In [ ]:
sns.set(font_scale=1.8)
plt.figure(figsize=(20,10))
df_behav=df_health[['Alcohol','animal_products','vegan products','manufactured product','other','Confirmed', 'Deaths',
       'Recovered']]
sns.heatmap(df_behav.corr(),annot=True,cmap="cool")

<h1 style="font-family:Georgia;font-size:200%;text-align:center">We note here that vegan products have an inverse relationship with the number of Confirmed cases (-5) , unlike animal products(+6), and this is a logical explanation for why evergreens restaurant was the most increase in sales rates during the Corona pandemic</h1>

<h1 style="color:white;background-color:darkviolet;font-family:Georgia;font-size:400%;text-align:center">PairGrid 


In [ ]:
sns.set(font_scale=1)
returns_fig = sns.PairGrid(df_behav)
#csutom plots easy to see 
returns_fig.map_upper(sns.kdeplot,cmap='cool_d')
returns_fig.map_lower(plt.scatter,color='purple')
returns_fig.map_diag(plt.hist,bins=40)


<img src="https://media.giphy.com/media/LMQnpxZ04RVmCn7eeE/giphy.gif">

<h1 style="background-color:skyblue;font-family:Georgia;font-size:350%;text-align:center">Conclusion</h1>

## 1- During The corona pandemic, The Most were eating organic food

    
    
## 2- Restaurants That rely on gatherings have faced significant losses 

    
    
## 3- Number of branches forms a big difference in sales, it helps to provide Quick Services 
  
    
    
## 4- It is worth relying on franchising for its high potential in achieving high sale


## 5- Going Vegan Can Reduce Severity Of COVID-19
